In [20]:
import numpy as np
import math

In [ ]:
# Load paper_edges processed by dataPreprocessing notebook. 
paper_edges = np.load("paper_edges_by_fields_only_in_dataset.npy", allow_pickle=True)
edges = paper_edges[()]['Computer Science']

In [5]:
len(edges)

46273

In [6]:
# Load paper nodes processed by dataPreprocessing notebook.
paper_nodes = np.load("paper_nodes_s2.npy", allow_pickle=True)[()]
nodes = paper_nodes['Computer Science']

In [8]:
len(nodes)

7642

In [11]:
# Turn nodes into a tuple (number index, node id)
numbered_nodes = []
for i in range(len(nodes)):
    numbered_nodes.append((i, nodes[i]))

In [12]:
# Turn nodes into a tuple (number index, node id)
num_node_map = {}
for i, node in numbered_nodes:
    num_node_map[node] = i

In [14]:
# Transform edges from node ids to number indices. 
num_edge = []
for edge in edges:
    a = edge[0]
    b = edge[1]
    num_edge.append((num_node_map[a], num_node_map[b]))

In [16]:
len(num_edge)

46273

In [29]:
numbered_nodes

[(0, '6e4eddf4d6671c37537bb5d1c9623353b62e8531'),
 (1, 'c8b8bc2a39da2f85a0526053872a6a008d3658db'),
 (2, 'fcb7e6cf3b4c349b729fa0444bbb5ac1f99c3f3a'),
 (3, '102e50c2c9a8e3d00de64a26759916c926fa3db6'),
 (4, '01f3fdbc51a44f4c39c23580f766375851fa40b8'),
 (5, '2472d4418db625fbbcbac22c38deb8e89635f085'),
 (6, 'bf20655839c1e8cfb5b27be8c76119d0ee7570ae'),
 (7, '030dd6b28b0c55dbb411f1eb8c280f5042969260'),
 (8, '268b37db3acb544de18348575b051a18e32033ab'),
 (9, '6c2b28f9354f667cd5bd07afc0471d8334430da7'),
 (10, 'e33bd10b340c0dfc87832d146619d7f857fe4708'),
 (11, 'eb3d0cbfeb8c6f9fa86bd212f558155e029c740e'),
 (12, 'cb2115a6765e8484830865b8ad5e6cc5dd29b48d'),
 (13, '7a6eb8341b607001d56f3789da93e1bc1a2ace68'),
 (14, 'f69cda1175c3ddf2d4e3f3789a6c743b1059ebb2'),
 (15, '90185c774a34bbbb303e58cbc7cd1cd16c6862bd'),
 (16, 'b338807be9f1e48de7dd9ece1834f844c46fdb90'),
 (17, 'b4f60b9c34b6eac774d63e1c9d787998505facd6'),
 (18, 'a35dee3fad164fe33f85e353a8c7a14e925aab25'),
 (19, 'b24ccf48da0a47d8dce36de63b541e7411

In [37]:
import pandas as pd
df = pd.DataFrame(numbered_nodes, columns=['ID','domain name'])
indegree = []
outdegree = []
# Get indegrees and outdegrees from the dict in the page rank result
# {"node": indegree, outdegree, set for all inward nodes}
for k, v in PR[0].items():
    indegree.append(v[0])
    outdegree.append(v[1])
df['in-links'] = indegree
df['out-links'] = outdegree
df.head()

,ID,domain name,in-links,out-links
0,0,6e4eddf4d6671c37537bb5d1c9623353b62e8531,4,0
1,1,c8b8bc2a39da2f85a0526053872a6a008d3658db,0,32
2,2,fcb7e6cf3b4c349b729fa0444bbb5ac1f99c3f3a,16,12
3,3,102e50c2c9a8e3d00de64a26759916c926fa3db6,9,2
4,4,01f3fdbc51a44f4c39c23580f766375851fa40b8,0,14


In [38]:
# The proportion of paper with no in-links (i.e., source papers);
l_df = df.shape[0]
prop_source = df['in-links'].value_counts()[0] / l_df
# The proportion of papers with no out-links (i.e., sink papers);
prop_sink = df['out-links'].value_counts()[0] / l_df
print("Proportion of websites with no in-links is {:2f}\n".format(prop_source))
print("Proportion of websites with no out-links is {:2f}".format(prop_sink))

Proportion of websites with no in-links is 0.387595

Proportion of websites with no out-links is 0.241036


In [44]:
def get_perplexity(PR):
    H = 0.0;
    for i in range(len(PR)):
        H += PR[i] * math.log2(PR[i])
    H *= -1
    return pow(2, H)

def page_rank(nodes, edges, threshold=1):
    d = 0.85
    N = len(nodes)
    page_rank = []
    PR = [1.0 / N] * N
    newPR = [0] * N
    P = [nodes[i][0] for i in range(N)]      # the set of all papers
    M = [set() for i in range(N)]      # the set of papers that cite paper p
    O = [set() for i in range(N)]     # the set of papers cited by paper p
    L = [0] * N      # the number of references of paper p
    S = []      # the set of sink nodes, papers that have no references which exist in the dataset.
    
    for start, end in edges:
        M[end].add(start)
        O[start].add(end)
    
    for i in range(N):
        L[i] = len(O[i])
        if L[i] == 0:
            S.append(i)

    perplexity = get_perplexity(PR)
    while True:
        sinkPR = 0.0
        for p in S:
            sinkPR += PR[p]
        for p in P:
            newPR[p] = (1 - d) / N
            newPR[p] += d * sinkPR / N
            for q in M[p]:
                newPR[p] += d * PR[q] / L[q]
        for p in P:
            PR[p] = newPR[p]
            
        new_perplexity = get_perplexity(PR)
        if abs(new_perplexity - perplexity) < threshold:
            break
        perplexity = new_perplexity
        
    for i in range(N):
        page_rank.append([str(nodes[i][0]), nodes[i][1], PR[i], len(M[i]), L[i]])
    
    page_rank.sort(key=lambda x: x[2], reverse=True)
    
    df_pr = pd.DataFrame(page_rank, columns =['Node No.', 'Node Name', 'PageRank', 'No. in-links', 'No. out-links']) 
#     print(df.head(10))
    
    return page_rank, df_pr
  
# Run until perplexity changes by less than 1
PR, df_pr = page_rank(numbered_nodes, num_edge, 1)

In [45]:
df_pr.head()

,Node No.,Node Name,PageRank,No. in-links,No. out-links
0,147,49af3e80343eb80c61e727ae0c27541628c7c5e2,0.010027,93,0
1,4798,c4775e2f0d27e8be4aae7b5b5c2560b96ce2eb58,0.007827,10,0
2,5438,bdede1e17c947540b50e6e2db9e8467ddc6e7336,0.005906,85,0
3,3823,0b44fcbeea9415d400c5f5789d6b892b6f98daff,0.005645,118,1
4,5936,807c1f19047f96083e13614f7ce20f2ac98c239a,0.005563,107,0


In [62]:
# Check mean PR scores and make sure that PR scores add to one. 
df_pr['PageRank'].mean(), df_pr['PageRank'].sum()

(0.00013085579691181722, 0.9999999999999986)

In [63]:
# Map PR scores to node ids. 
pr_map = {}
for index, row in df_pr.iterrows():
    pr_map[row['Node Name']] = row['PageRank']

In [46]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts=["http://localhost:9200"])

In [48]:
# Search function for the basic match. 
# Uses 'paperAbstract' and 'title', which should sufficiently represent the content of a paper.
# Will experiment with full text later. 
def search_BM25(query):
    MAX_SIZE = 20
    payload = {
        "multi_match": {
                "query": query,
                "fields": ['paperAbstract','title']
            }
            
        }
    resp = es.search(index="s2_doc", query=payload, size=MAX_SIZE)
    result = []
    # Extract info needed for display. 
    for item in resp['hits']['hits']:
        result.append(item['_id'])
    return result

In [66]:
import math
def calculate_nDCG(result, query_id, query_docno_gain):
    ideal_results = query_docno_gain[query_id]
    ideal_DCG = 0
    rank = 1
    for k, v in ideal_results.items():
        if rank == 1:
            ideal_DCG += v
        else:
            ideal_DCG += v / math.log2(rank)
        rank += 1
    if result[0] in ideal_results:
        nDCG = ideal_results[result[0]]
    else:
        nDCG = 0
    for i in range(1, len(result)):
        if result[i] in ideal_results:
            nDCG += ideal_results[result[i]] / math.log2(i + 1)
    nDCG /= ideal_DCG        
    return nDCG

def final_result(queries, query_docno_gain, pr_map):
    final_score = 0
    for k,v in queries.items():
        query_id = k
        query = v
        result = search_BM25(query)
        result = rerank(20, result, pr_map, 0.0001309)
        nDCG = calculate_nDCG(result, k, query_docno_gain)
        final_score += nDCG
    return final_score / 100

In [65]:
# Rerank with PR scores
def rerank(n, result, pr_map, average_pr):
    docno_pr = [('', 0) for _ in range(n)]
    for i in range(len(result)):
        docno = result[i]
        if docno in pr_map:
            pr = pr_map[docno]
        else:
            pr = average_pr
        docno_pr[i] = (docno, pr)
    docno_pr = sorted(docno_pr, key=lambda x : x[1])
    result = []
    for item in docno_pr:
        result.append(item[0])
    return result
    

In [50]:
# Ideal ranking results with relevance scores. 
import numpy as np
query_docno_gain = np.load("query_docno_gain.npy", allow_pickle=True)[()]

In [51]:
query_text = np.load("s2_query.npy", allow_pickle=True)[()]

In [56]:
# Remove quries not about CS. 
keys_to_remove = [6, 7, 14, 44, 63, 64, 65, 69, 71, 87, 88, 92, 94, 97, 98]

In [58]:
for key in keys_to_remove:
    del query_docno_gain[key]

In [69]:
for key in keys_to_remove:
    del query_text[key]

In [70]:
import time 
start = time.time()
final_score = final_result(query_text, query_docno_gain, pr_map)
end = time.time()
total_time = end - start

In [71]:
final_score

0.3966595426728006

In [72]:
total_time

2.0706350803375244